In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import pprint
import io

import pymongo
import json
import pandas as pd
from IPython import display

import climatools.lblnew.setup_overlap as setup_overlap
import climatools.lblnew.runrecord_overlap as runrecord_overlap
import climatools.lblnew.pipeline as pipeline

In [3]:
client = pymongo.MongoClient('localhost', 27017, connect=False)

In [4]:
db = client.lblnew

In [5]:
collection = db.overlap_lw

In [12]:
def make_doc(param=None):
    '''
    Make a Mongodb document for an lblnew-overlap run.
    
    Parameters
    ----------
    param: dict
        lblnew-overlap input parameters.
    doc: dict
        lblnew-overlap input (param) and output (flux and cooling rate)
    '''
    dir_fortran = pipeline.get_dir_case(param=param, 
                                        setup=setup_overlap)
    
    fnames = [n for n in os.listdir(dir_fortran) if n.endswith('.dat')]
    
    s_outputs = []
    for fname in fnames:
        with open(os.path.join(dir_fortran, fname),
                  mode='r', encoding='utf-8') as f:
            s = f.read()
            
        s_outputs.append(s)
        
    doc = {fname.split('.')[0]: s
           for fname, s in zip(fnames, s_outputs)}
    doc['param'] = param
    
    return doc    

def makeq(param=None):
    q = {}
    for n, v in param.items():
        if n == 'molecule' and type(v) == dict:
            for mol, conc in v.items():
                q[f'param.molecule.{mol}'] = conc
        else: 
            q[f'param.{n}'] = v
    return q    

In [7]:
params = runrecord_overlap.params()

In [20]:
docs = (make_doc(param=param) for param in params)
collection.insert_many(docs)

In [8]:
collection.count()

60